<a href="https://colab.research.google.com/github/mjgpinheiro/Econophysics/blob/main/Ergontropic_stocktrading3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import numpy as np
import yfinance as yf
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

# Define system variables
ticker = 'AAPL'
start_date = '2023-03-23'
end_date = '2023-03-24'
window_size = 30
stop_loss_pct = 0.05
investment_size = 10000

# Download stock price data
df_price = yf.download(ticker, start=start_date, end=end_date)

# Define system variables
api_key = '7f4f6db648674bb8afbcdb7567c7b91d'

# Load news data from NewsAPI
url = f"https://newsapi.org/v2/everything?q={ticker}&from={start_date}&to={end_date}&sortBy=popularity&apiKey={api_key}"
r = requests.get(url)
data = r.json()['articles']

# Convert news data to a pandas DataFrame
data_df = pd.DataFrame(data)
data_df['publishedAt'] = pd.to_datetime(data_df['publishedAt']).dt.date

# Rename columns
data_df = data_df.rename(columns={'publishedAt': 'Date', 'title': 'Headlines'})

# Clean news data
df_news = data_df[['Date', 'Headlines']]
df_news.set_index('Date', inplace=True)

# Calculate sentiment
df_news.loc[:, 'Sentiment'] = 0

# Calculate energy analogue
df_news.loc[:, 'Energy'] = np.log(len(df_news.Headlines.unique()))

# Calculate entropy analogue
df_news.loc[:, 'Entropy'] = -np.sum(df_news['Sentiment'] * np.log(df_news['Sentiment'])) / len(df_news)

# Merge news data with stock price data
merged_data = pd.merge(df_price, df_news, left_index=True, right_index=True, how='outer')
merged_data['Sentiment'].fillna(0, inplace=True)
merged_data.dropna(inplace=True)

# Prepare features and target
features = merged_data[['Energy', 'Entropy', 'Sentiment']]
target = merged_data['Adj Close'].pct_change().shift(-1)
features.dropna(inplace=True)
target.dropna(inplace=True)
features = features[:-1]
# Scale features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Train neural network
nn = MLPRegressor(hidden_layer_sizes=(10, 10), activation='relu', solver='adam', max_iter=5000, random_state=0)
nn.fit(scaled_features, target)

# Get last day's features
last_day_features = scaled_features[-1].reshape(1, -1)

# Predict next day's return
predicted_return = nn.predict(last_day_features)

# Calculate stop-loss
stop_loss = df_price['Adj Close'][-1] * (1 - stop_loss_pct)

# Make trading decision based on predicted return and stop-loss
if predicted_return > 0 and df_price['Adj Close'][-1] < stop_loss:
    trade = 'Buy'
    shares = investment_size / df_price['Adj Close'][-1]
elif predicted_return < 0:
    trade = 'Sell'
    shares = investment_size / df_price['Adj Close'][-1]
else:
    trade = 'Hold'
    shares = 0

# Print trading decision
print(f'Trade: {trade}, Shares: {shares:.2f}, Investment Size: {investment_size:.2f}')



